<img src="../../data/images/gempy_logo.png" />

# <center> From Maps to Models - Tutorials for structural geological modeling using GemPy and GemGIS</center>

# Post-Processing Example 2 - Extracting virtual boreholes from Meshes

This post-processing example shows how to extract vertical and deviated boreholes from PyVista meshes. This means getting the XYZ position where a well intersects with the respective mesh. 

## Import GemGIS

If you have installed ``GemGIS`` via pip or conda, you can import ``GemGIS`` like any other package. If you have downloaded the repository, append the path to the directory where the ``GemGIS`` repository is stored and then import ``GemGIS``. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import gemgis as gg

## Importing Libraries

In [ ]:
import numpy as np
import pyvista as pv
import pandas as pd

## Loading and Plotting Meshes

The meshes and colors of the meshes were exported from example 5 and will now be used for this example.



In [ ]:
mesh1 = pv.read('../../data/postproc2_boreholes/mesh1.vtk')
mesh1

In [ ]:
mesh2 = pv.read('../../data/postproc2_boreholes/mesh2.vtk')
mesh2

In [ ]:
colors = f = open('../../data/postproc2_boreholes/colors.txt', "r")
colors = colors.read().split("\n")[:2]
colors 

In [ ]:
p = pv.Plotter(notebook=True)

p.add_mesh(mesh1, color=colors[0])
p.add_mesh(mesh2, color=colors[1])

p.show_grid(color= 'black')
p.set_background('white')
p.show()

## Extracting intersections between vertical boreholes and meshes

Firstly, a PolyData set is created and visualized. The first attempt shows a straight vertical well. We define the X coordinate to be 1000 m and the Y coordinate to be 1500 m. The well starts at a altitude and is drilled to an altitude of -100 m.

In [ ]:
well = pv.Line((1000,1500, 800), (1000,1500, -100))
well_tube = well.tube(radius=20)
well_tube

In [ ]:
p = pv.Plotter(notebook=True)

p.add_mesh(mesh1, color=colors[0])
p.add_mesh(mesh2, color=colors[1])
p.add_mesh(well_tube, color='white')
p.show_grid(color= 'black')
p.set_background('white')
p.show()

In [ ]:
well_tops = gg.utils.create_virtual_profile(names_surfaces=['A', 'B'],
                                            surfaces=[mesh1, mesh2],
                                            borehole=well)
well_tops

## Extracting intersections between deviated boreholes and meshes

The second well represents a deviated well that intersects the meshes multiple times

In [ ]:
points = np.array(([2500,1500, 800],
                   [2500,1500, 400],
                   [1000, 1500, 400],
                   [0, 1500, 400],
                  ))

# From https://docs.pyvista.org/examples/00-load/create-spline.html
def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points) - 1, 3), 2, dtype=np.int_)
    cells[:, 1] = np.arange(0, len(points) - 1, dtype=np.int_)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int_)
    poly.lines = cells
    return poly


well = lines_from_points(points)

well_tube = well.tube(radius=20)
well_tube

In [ ]:
p = pv.Plotter(notebook=True)

p.add_mesh(mesh1, color=colors[0])
p.add_mesh(mesh2, color=colors[1])
p.add_mesh(well_tube, color='white')
p.show_grid(color= 'black')
p.set_background('white')
p.show()

In [ ]:
well_tube.points[0]

In [ ]:
from typing import Union

def create_virtual_profile(names_surfaces: list,
                           surfaces: list,
                           borehole_top: Union[np.ndarray, list],
                           borehole_bottom: Union[np.ndarray, list],
                           first_point: bool = False):
    """ Function to filter and sort the resulting well tops
    Parameters:
    ___________
        names_surfaces: list
            List of the names of the calculated GemPy surfaces
        surfaces: list
            List of calculated GemPy surfaces
        borehole_top:
            Coordinates of the top of the well
        borehole_bottom:
            Coordinates of the bottom of the well
        first_point: bool
            Returns intersection of first point only
    """

    # Extracting well tops
    well_tops = gg.utils.ray_trace_multiple_surfaces(surfaces=surfaces,
                                            borehole_top=borehole_top,
                                            borehole_bottom=borehole_bottom)

    # Creating dict from names and well tops
    well_dict = dict(zip(names_surfaces, well_tops))

    # Removing empty entries
    for key in well_dict.copy():
        if not well_dict[key][0].any():
            well_dict.pop(key)

    # Splitting layers if they have been encountered more than once
    for key in well_dict.copy():
        if len(well_dict[key][0]) > 1:
            for i in range(1, len(well_dict[key][0])):
                well_dict[key + '_%s' % str(i + 1)] = (well_dict[key][0][i].reshape(1, 3), well_dict[key][1][i])

    # Extracting only Z value from dict
    for key in well_dict.keys():
        well_dict[key] = well_dict[key][0][0]

    # Sorting well dict
#     well_dict = dict(sorted(well_dict.items(), key=lambda item: item[1], reverse=True))

    return well_dict

In [ ]:
well_tops = [create_virtual_profile(names_surfaces=['A', 'B'],
                                            surfaces=[mesh1, mesh2],
                                            borehole_top=well.points[i],
                                            borehole_bottom=well.points[i+1]) for i in range(len(well.points)-1)]



well_tops_df = pd.concat([pd.DataFrame.from_dict(df, orient='index', columns=[ 'X', 'Y', 'Z']) for df in well_tops])

well_tops_df

<a id='licensing'></a>

## Licensing

Institute for Computational Geoscience, Geothermics and Reservoir Geophysics, RWTH Aachen University & Fraunhofer IEG, Fraunhofer Research Institution for Energy Infrastructures and Geothermal Systems IEG, Authors: Alexander Juestel. For more information contact: alexander.juestel(at)ieg.fraunhofer.de

All notebooks are licensed under a Creative Commons Attribution 4.0 International License (CC BY 4.0, http://creativecommons.org/licenses/by/4.0/). References for each displayed map are provided. Most of the maps originate from the books of [Powell (1992)](https://link.springer.com/book/9783540586074) and [Bennison (1990)](https://link.springer.com/book/10.1007/978-1-4615-9630-1). References for maps with unknown origin will gladly be added.